In [1]:
import torch
import torch.nn.functional as F
import urllib3
import numpy as np
import os, sys, random, shutil
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# download the names.text file from github
import urllib.request
url = 'https://raw.githubusercontent.com/karpathy/makemore/master/names.txt'
filename = os.path.basename(url)
urllib.request.urlretrieve(url, filename)
os.listdir()

['.ipynb_checkpoints', 'Backprop_andrej_karpathy.ipynb', 'names.txt']

In [3]:
os.path.basename(url)

'names.txt'

In [4]:
# reald in all words
words = open('names.txt', 'r').read().splitlines()
print('no. of names', len(words))
print('longest name:', max(len(w) for w in words))
print('sample names:\n', words[:5])

no. of names 32033
longest name: 15
sample names:
 ['emma', 'olivia', 'ava', 'isabella', 'sophia']


In [6]:
#build the vocabulary of characters and mappings to/from integers

chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.']=0
itos = {i:s for s, i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [8]:
# build the cataset

block_size = 3 # context length : no. of characters to be considered to predict the next char

def build_dataset(words):
    X, Y = list(), list()

    for w in words:
        context = [0]*block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y


import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


<h4> Utility function to compare gradients</h4>

In [45]:
def cmp(s, dt, t):
    ex = torch.all(dt==t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt-t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: { str(app):5s} | maxdiff: {maxdiff}')

In [14]:
Xtr.shape

torch.Size([182625, 3])

In [37]:
n_embd = 10 # the dimensionality of the character embedding vector
n_hidden = 64

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embd), generator=g)

# layer 1
W1 = torch.randn(n_embd*block_size, n_hidden) # W1 : (30, 64)
b1 = torch.randn(n_hidden, generator=g) # b1 : (64, 1)

# layer 2
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1

# BatchNorm parameters
bngain = torch.rand((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

parameters = [ C, W1, b1, W2, b2, bngain, bnbias ]

print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

4137


In [38]:
batch_size = 32
n = batch_size

# contruct a mini-batch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X, Y

In [39]:
C.shape

torch.Size([27, 10])

tensor(3.3487, grad_fn=<NegBackward0>)

In [ ]:
h.shape, W2.shape, b2.shape

Xb.shape

dloss = 1

dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1/n
cmp('logprobs', dlogprobs, logprobs)


dprobs = 